In [1]:
#### Nomenclature
# B beginning of review
# L length of review (by characters)
# b beginning of unit (how many characters in does the highlight begin)
# l length of unit (by characters)
# categories assigned by c or k
# annotators identified by i or j
# sections identified by g or h (for annotators i and j respectively)
# gap or section defined by v (0 or 1 respectively)

In [2]:
# Phrases that have not been highlighted must have empty cells in Excel (not have 0s).
# Same applies to any "Category" cells corresponding to that phrase.

In [3]:
def krippendorff_alpha_u(xlsx_file_url, sheet_name):
    dictionary = store_annotation_data_in_dictionary(xlsx_file_url, sheet_name)
    DocDec = calculate_obsereved_and_expected_disagreement(dictionary[0], dictionary[1], dictionary[2])
    alphas = calculate_alpha(DocDec[0], DocDec[1])
    return alphas

In [4]:
def store_annotation_data_in_dictionary(xlsx_file_url, sheet_name):
    #### read data into data frame
    import pandas as pd
    df = pd.read_excel(xlsx_file_url, sheet_name=sheet_name)
    
    B = 0
    
    # determine maximum number of highlights per review
    n_highlights_max = 0
    for col in df.columns:
        if 'Phrase' in col:
            n_highlights_max += 1

    # determine maximum number of categories       
    n_categories = 0
    for i in range(1, n_highlights_max + 1):
        if df['Category' + str(i)].isnull().all():
            continue
        n_category = int(df['Category' + str(i)].max())
        if n_category > n_categories:
            n_categories = n_category
    n_categories += 1
    print('number_of_categories: ' + str(n_categories))

    annotation_data = [None]*n_categories
    L_dict = {}

    # populate dictionary with data from Excel file
    for category in range(0, n_categories):
        annotation_data[category] = {}
        n_coder = 0
        for index, row in df.iterrows():
            #skip row if does not contain annotation
            if row['Phrase1'] == '0' or row['Phrase1'] == 0 or row['Phrase1'] == '':
                continue

            # add review to dictionary
            if 'review' + str(row.ID) in annotation_data[category].keys():
                n_coder += 1
            else:
                n_coder = 0 
                annotation_data[category]['review' + str(row.ID)] = {}
            annotations = []
            
            # determine number of annotations
            for i in range(1,n_highlights_max + 1):
                #remove apostrophe at the beginning of a phrase if present
                #if isinstance(row['Phrase'+str(i)], str):
                if str(row['Phrase'+str(i)])[0] == "'" and len(str(row['Phrase'+str(i)])) > 1:
                    if str(row['Phrase'+str(i)])[1] == '-' or str(row['Phrase'+str(i)])[1] == '+' or str(row['Phrase'+str(i)])[1] == '=' or str(row['Phrase'+str(i)])[1] == '$':
                        row['Phrase'+str(i)] = str(row['Phrase'+str(i)])[1:]
                #remove space at the end of a phrase if present
                if str(row['Phrase'+str(i)]).endswith(' '):
                    row['Phrase'+str(i)] = str(row['Phrase'+str(i)])[:-1]
                #standardize 0 format 
                elif row['Phrase'+str(i)] == '0' or row['Phrase'+str(i)] == '0.0' or row['Phrase'+str(i)] == 0.0:
                    row['Phrase'+str(i)] = 0  

                if row['Category' + str(i)] == category:
                    annotations.append(row['Phrase'+str(i)])
                else:
                    annotations.append(0)

            n_highlights = n_highlights_max - annotations.count(0)
            #n_sections = 2*(n_highlights)+1

#             last_highlight_index = [index for index, item in enumerate(annotations) if item != 0]
#             if len(last_highlight_index) != 0:
#                 last_highlight_index = last_highlight_index[-1]

            review = row.Review
            L = len(review)
            L_dict['review' + str(row.ID)] = L
            b = [B]
            l = []
            
            # make b vector
            for i in range(1, n_highlights_max + 1):
                if row['Category' + str(i)] == category:
                    b += [review.find(str(row['Phrase'+str(i)])), review.find(str(row['Phrase'+str(i)])) + len(str(row['Phrase'+str(i)]))]

            # make l vector
            for i in range(len(b) - 1) : 
                l.append(abs(b[i] - b[i + 1]))
            l.append(L - b[-1])

            # make v vector
            v = ([0, 1] * n_highlights) + [0]

            if b == [B]:
                continue

            
            #adjust if there are multiple adjacent highlights for the same category
            for j in range (0, l.count(1)):
                if len(b) <= 3:
                    continue           
                for i in range(0, len(l)):

                    if i >= len(l)-2:
                        break
                    if l[i] == 1:
                        del b[i+1]
                        del b[i]
                        l[i-1] = l[i-1] + l[i+1] + l[i]
                        del l[i+1]
                        del l[i]
                        del v[i+2]
                        del v[i+1]


            #store data properties in dictionary
            annotation_data[category]['review' + str(row.ID)]['coder'+str(n_coder)]={}
            annotation_data[category]['review' + str(row.ID)]['coder'+str(n_coder)]['b'] = b
            annotation_data[category]['review' + str(row.ID)]['coder'+str(n_coder)]['l'] = l
            annotation_data[category]['review' + str(row.ID)]['coder'+str(n_coder)]['v'] = v


    print('annotation_data: ' + str(annotation_data))
    print('length_of_reviews: ' + str(L_dict))
    
    return annotation_data, n_categories, L_dict

In [5]:
# make doc and dec into dictionary
def calculate_obsereved_and_expected_disagreement(annotation_data, n_categories, L_dict):
    Doc = {}
    Dec = {}
    for category in range(0, n_categories):

        Doc['category' + str(category)] = {}
        Dec['category' + str(category)] = {}
        for review in annotation_data[category]:
            L = L_dict[review]
            b = []
            l = []
            v = []
            d = []
            for coder in annotation_data[category][review]:
                i = 0
                # extract annotation propoerties into lists
                for annotation in annotation_data[category][review][coder]:

                    if i == 0:
                        b0 = annotation_data[category][review][coder][annotation]
                        b += b0
                        i += 1
                        continue
                    if i == 1:
                        l0 = annotation_data[category][review][coder][annotation]
                        l += l0
                        i += 1
                        continue
                    if i == 2:
                        v0 = annotation_data[category][review][coder][annotation]
                        v += v0
                        i = 0

                        # loop through other annotations
                        for other_coder in annotation_data[category][review]:
                            for other_annotation in annotation_data[category][review][other_coder]:
                                if i == 0:
                                    b1 = annotation_data[category][review][other_coder][other_annotation]
                                    i += 1
                                    continue
                                if i == 1:
                                    l1 = annotation_data[category][review][other_coder][other_annotation]
                                    i += 1
                                    continue
                                if i == 2:
                                    v1 = annotation_data[category][review][other_coder][other_annotation]
                                    i = 0

                                    if b1 == b0 and l1 == l0 and v1 == v0:
                                        d.append(0)
                                        continue

                                    # calculate difference d between two annotations
                                    b0_limit = len(b0)-1
                                    b1_limit = len(b1)-1
                                    #d = []
                                    vec = []
                                    vec.append(b0)
                                    vec.append(b1)

                                    if len(b0)>len(b1):
                                        for index, x in enumerate(b0):
                                            if index >= b1_limit:
                                                # if both are units
                                                if v1[b1_limit] == 1 and v0[index] == 1:
                                                    # if there is no overlap
                                                    if (b0[index] + l0[index]) < b1[b1_limit] or b1[b1_limit] + l1[b1_limit]<b0[index]:
                                                        d.append(l0[index]**2 + l1[b1_limit]**2)
                                                    # if there is overlap
                                                    if (b0[index] <= b1[b1_limit] + l1[b1_limit] <= b0[index] + l0[index]) or (b1[b1_limit] <= b0[index] + l0[index] <= b1[b1_limit] + l1[b1_limit]):
                                                        d.append((b0[index]-b1[b1_limit])**2+(b0[index]+l0[index]-b1[b1_limit]-l1[b1_limit])**2)
                                                elif v1[b1_limit] == 1 and v0[index] == 0:
                                                    d.append(l1[b1_limit]**2)
                                                elif v1[b1_limit] == 0 and v0[index] == 1:
                                                    d.append(l0[index]**2)
                                                else:
                                                    d.append(0)
                                            else:
                                                # if both are units
                                                if v1[index] == 1 and v0[index] == 1:
                                                    # if there is no overlap
                                                    if (b0[index] + l0[index]) < b1[index] or b1[index] + l1[index]<b0[index]:
                                                        d.append(l0[index]**2 + l1[index]**2)
                                                    # if there is overlap
                                                    if (b0[index] <= b1[index] + l1[index] <= b0[index] + l0[index]) or (b1[index] <= b0[index] + l0[index] <= b1[index] + l1[index]):
                                                        d.append((b0[index]-b1[index])**2+(b0[index]+l0[index]-b1[index]-l1[index])**2)
                                                elif v1[index] == 1 and v0[index] == 0:
                                                    d.append(l1[index]**2)
                                                elif v1[index] == 0 and v0[index] == 1:
                                                    d.append(l0[index]**2)
                                                else:
                                                    d.append(0)
                                    else:
                                        for index, x in enumerate(b1):
                                            if index >= b0_limit:
                                                # if both are units
                                                if v1[index] == 1 and v0[b0_limit] == 1:
                                                    # if there is no overlap
                                                    if (b0[b0_limit] + l0[b0_limit]) < b1[index] or b1[index] + l1[index]<b0[b0_limit]:
                                                        d.append(l0[b0_limit]**2 + l1[index]**2)
                                                    # if there is overlap
                                                    if (b0[b0_limit] <= b1[index] + l1[index] <= b0[b0_limit] + l0[b0_limit]) or (b1[index] <= b0[b0_limit] + l0[b0_limit] <= b1[index] + l1[index]):
                                                        d.append((b0[b0_limit]-b1[index])**2+(b0[b0_limit]+l0[b0_limit]-b1[index]-l1[index])**2)
                                                elif v1[index] == 1 and v0[b0_limit] == 0:
                                                    d.append(l1[index]**2)
                                                elif v1[index] == 0 and v0[b0_limit] == 1:
                                                    d.append(l0[b0_limit]**2)
                                                else:
                                                    d.append(0)
                                            else:
                                                # if both are units
                                                if v1[index] == 1 and v0[index] == 1:
                                                    # if there is no overlap
                                                    if (b0[index] + l0[index]) < b1[index] or b1[index] + l1[index]  < b0[index]:
                                                        d.append(l0[index]**2 + l1[index]**2)
                                                    # if there is overlap
                                                    if (b0[index] <= b1[index] + l1[index] <= b0[index] + l0[index]) or (b1[index] <= b0[index] + l0[index] <= b1[index] + l1[index]):
                                                        d.append((b0[index]-b1[index])**2+(b0[index]+l0[index]-b1[index]-l1[index])**2)
                                                elif v1[index] == 1 and v0[index] == 0:
                                                    d.append(l1[index]**2)
                                                elif v1[index] == 0 and v0[index] == 1:
                                                    d.append(l0[index]**2)
                                                else:
                                                    d.append(0)



            # append doc for each review
            m = len(annotation_data[category][review])

            if m == 1 or m == 0:
                continue

            Doc['category' + str(category)][review] = (sum(d)/(m*(m-1)*L**2))

            # Calculate Dec
            n=v.count(1)
            numerator = 0
            second_term_d=0
            second_term_n = 0        
            for index, x in enumerate(b):
                if v[index] == 1:
                    second_term_d += l[index]*(l[index]-1)
                    first_term_n = ((n-1)/3)*((2*(l[index]**3))-(3*l[index]**2)+l[index])
                    second_term_n = 0
                    for index_1, x_1 in enumerate(b):
                        if l[index_1] >= l[index] and v[index_1] ==0:
                            second_term_n += (l[index_1]-l[index]+1)
                    numerator += first_term_n + (l[index]**2)*second_term_n


            numerator = (2/L)*numerator
            first_term_d = m*L*(m*L-1)
            denominator = first_term_d - second_term_d

            # append dec for each review
            Dec['category' + str(category)][review] = (numerator / denominator)

    print('Doc_by_category: ' + str(Doc))
    print('Dec_by_category: ' + str(Dec))
    
    return Doc, Dec

In [6]:
def calculate_alpha(Doc, Dec):
    import numpy as np
    #### Alpha intercoder agreement
    alpha_category = {}
    alpha_combined = {}
    ## alpha for each category
    for category in Doc:
        for review in Doc[category]:
            if review in alpha_category.keys():
                pass
            else:
                alpha_category[review] = {}
            alpha_category[review][category] = 1 - (Doc[category][review]/Dec[category][review])
    print('alpha_by_category: ' + str(alpha_category))

    ## combined alpha
    alpha_combined = {}
    sum_Doc = {}
    sum_Dec = {}

    # calculate Doc and Dec sum
    for category in Doc:
        for review in Doc[category]:
            if review in sum_Doc.keys():
                pass
            else:
                sum_Doc[review] = 0
                sum_Dec[review] = 0
            sum_Doc[review] += Doc[category][review]
            sum_Dec[review] += Dec[category][review]

    # calculate alpha    
    for review in sum_Doc:
        if review in alpha_combined.keys():
            pass
        else:
            alpha_combined[review] = {}
        alpha_combined[review] = 1 - sum_Doc[review] / sum_Dec[review]
    print('alpha_combined: ' + str(alpha_combined))
    
    return alpha_category, alpha_combined

In [7]:
sustainability = 'Economic' # Social, Envrionmental, or Economic
output = krippendorff_alpha_u("/Users/ndehaibi/Desktop/Research/Study 2/IRR/IRR Test (" + str(sustainability) + ").xlsx", 'IRR')
alpha_combined = list((output[1].values()))
from matplotlib import pyplot as plt
bins = 10
arr = plt.hist(alpha_combined, bins)
plt.ylabel('Review Count')
plt.xlabel('Intercoder Agreement')
plt.title(str(sustainability) + ' Sustainability - Letters')
for i in range(bins):
    plt.text(arr[1][i],arr[0][i] + 1.2,str(arr[0][i]))
print(len(alpha_combined))
import statistics
print('mean: ' + str(statistics.mean(alpha_combined)))
print('std dev: ' + str(statistics.stdev(alpha_combined)))

number_of_categories: 2
annotation_data: [{'review1': {'coder1': {'b': [0, 75, 147], 'l': [75, 72, 651], 'v': [0, 1, 0]}, 'coder2': {'b': [0, 291, 370, 734, 798], 'l': [291, 79, 364, 64, 0], 'v': [0, 1, 0, 1, 0]}}, 'review2': {'coder2': {'b': [0, 10, 15], 'l': [10, 5, 558], 'v': [0, 1, 0]}}, 'review3': {'coder2': {'b': [0, 803, 951], 'l': [803, 148, 0], 'v': [0, 1, 0]}}, 'review4': {}, 'review5': {}, 'review6': {}, 'review7': {}, 'review8': {'coder0': {'b': [0, 25, 30], 'l': [25, 5, 470], 'v': [0, 1, 0]}}, 'review9': {'coder0': {'b': [0, 27, 113], 'l': [27, 86, 579], 'v': [0, 1, 0]}}, 'review10': {}, 'review11': {}, 'review12': {'coder1': {'b': [0, 63, 262], 'l': [63, 199, 442], 'v': [0, 1, 0]}, 'coder2': {'b': [0, 63, 262], 'l': [63, 199, 442], 'v': [0, 1, 0]}}, 'review13': {}, 'review14': {'coder1': {'b': [0, 109, 208], 'l': [109, 99, 551], 'v': [0, 1, 0]}, 'coder2': {'b': [0, 154, 208], 'l': [154, 54, 551], 'v': [0, 1, 0]}}, 'review15': {}, 'review16': {}, 'review17': {}, 'review18'

436
mean: -0.08834805495577137
std dev: 0.9052141554989043


In [65]:
########## DEBUGGING CODE BELOW ##########

In [10]:
### Example from Chapter 9 "Unitizing and coding coherent segments in continua"
L_dict={}
L_dict['review1']=76
n_categories = 5

annotation_data = [None]*n_categories
annotation_data[0] = {}
annotation_data[0]['review1'] = {}
annotation_data[0]['review1']['coder0'] = {}
annotation_data[0]['review1']['coder1'] = {}
annotation_data[0]['review1']['coder0']['b'] = [0,2,17,22,37] 
annotation_data[0]['review1']['coder0']['l'] = [2,15,5,15,39]
annotation_data[0]['review1']['coder0']['v'] = [0, 1, 0, 1, 0]
annotation_data[0]['review1']['coder1']['b'] = [0,2,17,58,63] 
annotation_data[0]['review1']['coder1']['l'] = [2,15,41,5,13]
annotation_data[0]['review1']['coder1']['v'] = [0, 1, 0, 1, 0]
annotation_data[1] = {}
annotation_data[1]['review1'] = {}
annotation_data[1]['review1']['coder0'] = {}
annotation_data[1]['review1']['coder0']['b'] = [0,58,68] 
annotation_data[1]['review1']['coder0']['l'] = [58,10,8]
annotation_data[1]['review1']['coder0']['v'] = [0, 1, 0]
annotation_data[2] = {}
annotation_data[2]['review1'] = {}
annotation_data[2]['review1']['coder1'] = {}
annotation_data[2]['review1']['coder1']['b'] = [0,58,68] 
annotation_data[2]['review1']['coder1']['l'] = [58,10,8]
annotation_data[2]['review1']['coder1']['v'] = [0, 1, 0]
annotation_data[3] = {}
annotation_data[3]['review1'] = {}
annotation_data[3]['review1']['coder0'] = {}
annotation_data[3]['review1']['coder1'] = {}
annotation_data[3]['review1']['coder0']['b'] = [0,70,75] 
annotation_data[3]['review1']['coder0']['l'] = [70,5,1]
annotation_data[3]['review1']['coder0']['v'] = [0, 1, 0]
annotation_data[3]['review1']['coder1']['b'] = [0,70,75]
annotation_data[3]['review1']['coder1']['l'] = [70,5,1]
annotation_data[3]['review1']['coder1']['v'] = [0, 1, 0]
annotation_data[4] = {}
annotation_data[4]['review1'] = {}
annotation_data[4]['review1']['coder1'] = {}
annotation_data[4]['review1']['coder1']['b'] = [0,45,48] 
annotation_data[4]['review1']['coder1']['l'] = [45,3,28]
annotation_data[4]['review1']['coder1']['v'] = [0, 1, 0]

print(annotation_data)


DocDec = calculate_obsereved_and_expected_disagreement(annotation_data, n_categories, L_dict)
alphas = calculate_alpha(DocDec[0], DocDec[1])


[{'review1': {'coder0': {'b': [0, 2, 17, 22, 37], 'l': [2, 15, 5, 15, 39], 'v': [0, 1, 0, 1, 0]}, 'coder1': {'b': [0, 2, 17, 58, 63], 'l': [2, 15, 41, 5, 13], 'v': [0, 1, 0, 1, 0]}}}, {'review1': {'coder0': {'b': [0, 58, 68], 'l': [58, 10, 8], 'v': [0, 1, 0]}}}, {'review1': {'coder1': {'b': [0, 58, 68], 'l': [58, 10, 8], 'v': [0, 1, 0]}}}, {'review1': {'coder0': {'b': [0, 70, 75], 'l': [70, 5, 1], 'v': [0, 1, 0]}, 'coder1': {'b': [0, 70, 75], 'l': [70, 5, 1], 'v': [0, 1, 0]}}}, {'review1': {'coder1': {'b': [0, 45, 48], 'l': [45, 3, 28], 'v': [0, 1, 0]}}}]
Doc_by_category: {'category0': {'review1': 0.0432825484764543}, 'category1': {}, 'category2': {}, 'category3': {'review1': 0.0}, 'category4': {}}
Dec_by_category: {'category0': {'review1': 0.06560657764939656}, 'category1': {}, 'category2': {}, 'category3': {'review1': 0.007718318141723022}, 'category4': {}}
alpha_by_category: {'review1': {'category0': 0.34027120408936007, 'category3': 1.0}}
alpha_combined: {'review1': 0.4097155132718

In [11]:
### Example from "Measuring the Reliability of Qualitative Text Analysis Data". 
L_dict = {}
L_dict['review1']=300
n_categories = 2
annotation_data = [None]*n_categories
annotation_data[0] = {}
annotation_data[0]['review1'] = {}
annotation_data[0]['review1']['coder0'] = {}
annotation_data[0]['review1']['coder1'] = {}
annotation_data[0]['review1']['coder0']['b'] = [150, 225, 295, 370, 400] 
annotation_data[0]['review1']['coder0']['l'] = [75, 70, 75, 30, 50]
annotation_data[0]['review1']['coder0']['v'] = [0, 1, 0, 1, 0]
annotation_data[0]['review1']['coder1']['b'] = [150, 220, 300, 355, 375, 400, 420] 
annotation_data[0]['review1']['coder1']['l'] = [70, 80, 55, 20, 25, 20, 30]
annotation_data[0]['review1']['coder1']['v'] = [0, 1, 0, 1, 0, 1, 0]
annotation_data[1] = {}
annotation_data[1]['review1'] = {}
annotation_data[1]['review1']['coder0'] = {}
annotation_data[1]['review1']['coder0']['b'] = [150, 180, 240, 300, 350] 
annotation_data[1]['review1']['coder0']['l'] = [30, 60, 60, 50, 100]
annotation_data[1]['review1']['coder0']['v'] = [0, 1, 0, 1, 0]
annotation_data[1]['review1']['coder1'] = {}
annotation_data[1]['review1']['coder1']['b'] = [150, 180, 240, 300, 350] 
annotation_data[1]['review1']['coder1']['l'] = [30, 60, 60, 50, 100]
annotation_data[1]['review1']['coder1']['v'] = [0, 1, 0, 1, 0]
print(annotation_data)


DocDec = calculate_obsereved_and_expected_disagreement(annotation_data, n_categories, L_dict)
alphas = calculate_alpha(DocDec[0], DocDec[1])


[{'review1': {'coder0': {'b': [150, 225, 295, 370, 400], 'l': [75, 70, 75, 30, 50], 'v': [0, 1, 0, 1, 0]}, 'coder1': {'b': [150, 220, 300, 355, 375, 400, 420], 'l': [70, 80, 55, 20, 25, 20, 30], 'v': [0, 1, 0, 1, 0, 1, 0]}}}, {'review1': {'coder0': {'b': [150, 180, 240, 300, 350], 'l': [30, 60, 60, 50, 100], 'v': [0, 1, 0, 1, 0]}, 'coder1': {'b': [150, 180, 240, 300, 350], 'l': [30, 60, 60, 50, 100], 'v': [0, 1, 0, 1, 0]}}}]
Doc_by_category: {'category0': {'review1': 0.014444444444444444}, 'category1': {'review1': 0.0}}
Dec_by_category: {'category0': {'review1': 0.05322177985498048}, 'category1': {'review1': 0.04897856580891525}}
alpha_by_category: {'review1': {'category0': 0.728598996805389, 'category1': 1.0}}
alpha_combined: {'review1': 0.8586654051841702}


In [22]:
### Checking code with hand calculations
import pandas as pd
df = pd.read_excel("/Users/ndehaibi/Desktop/Research/Study 2/IRR/IRR Test (social).xlsx", sheet_name='IRR_mini')
L = len(df['Review'][0])
review = df['Review'][0]
print(L)
B = 0

b11 = 0
l11 = review.find(df['Phrase1'][0])
b12 = review.find(df['Phrase1'][0])
l12 = len(df['Phrase1'][0])
b13 = b12 + l12
l13 = L - b13

b21 = 0
l21 = review.find(df['Phrase1'][1])
b22 = review.find(df['Phrase1'][1])
l22 = len(df['Phrase1'][1])
b23 = b22 + l22
l23 = L - b23

b31 = 0
l31 = review.find(df['Phrase1'][2])
b32 = review.find(df['Phrase1'][2])
l32 = len(df['Phrase1'][2])
b33 = b32 + l32
l33 = L - b33

print(b11)
print(l11)
print(b12)
print(l12)
print(b13)
print(l13)
print(b21)
print(l21)
print(b22)
print(l22)
print(b23)
print(l23)
print(b31)
print(l31)
print(b32)
print(l32)
print(b33)
print(l33)

144
0
12
12
43
55
89
0
95
95
27
122
22
0
95
95
27
122
22
